In [13]:
import os
import json

def read_json(fn):
    with open(fn, 'r') as f:
        return json.load(f)
    
ROOT_DATASET_DIR = '/datasets/COCO'
IMAGES_DIR = os.path.join(ROOT_DATASET_DIR, 'images')
ANNOTATIONS_JSON_PATH = os.path.join(ROOT_DATASET_DIR, 'annotations/instances_val2017.json')
COUNT = 100

annotations = read_json(ANNOTATIONS_JSON_PATH)

In [14]:
import random
ids = random.sample([x['id'] for x in annotations['images']], COUNT)

annotations['images'] = [x for x in annotations['images'] if x['id'] in ids]   
annotations['annotations'] = [x for x in annotations['annotations'] if x['image_id'] in ids]

len(annotations['images'])

100

In [17]:
with open(f"temp_{COUNT}.json", 'w') as f:
    json.dump(annotations, f, indent=4)

### Arch checker

In [1]:
import torch
from aimet_torch.arch_checker.arch_checker import ArchChecker
from mmdet.apis import DetInferencer

CONFIG_PATH = 'rtmdet_tiny_8xb32-300e_coco.py'
WEIGHTS_PATH = 'rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth'

dummy_input = torch.rand(1, 3, 640, 640)
model = DetInferencer(model=CONFIG_PATH, weights=WEIGHTS_PATH)

type(model.model)

2024-08-02 17:00:47,852 - root - INFO - AIMET
Loads checkpoint by local backend from path: rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std

08/02 17:00:52 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.


/home/shayaan/miniconda3/envs/aimet2/lib/python3.10/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


mmdet.models.detectors.rtmdet.RTMDet

In [2]:
from aimet_torch.quantsim import QuantizationSimModel, QuantScheme
from aimet_torch.model_preparer import prepare_model

sim_model = prepare_model(model.model)
sim2 = prepare_model(sim_model)
sim3 = prepare_model(sim2)
type(sim_model), type(sim2)

In [4]:
ArchChecker.check_model_arch(model.model, dummy_input, "./")

(((((((((((CONSTRUCT GRAPH)))))))))))
<class 'torch.jit._trace.TopLevelTracedModule'>
<class 'mmdet.models.detectors.rtmdet.RTMDet'>


KeyError: '1'